# **Final Project Task 1 - Census Data Preprocess**

Requirements

- Target variable specification:
    - The target variable for this project is hours-per-week. 
    - Ensure all preprocessing steps are designed to support regression analysis on this target variable.
- Encode data  **3p**
- Handle missing values if any **1p**
- Correct errors, inconsistencies, remove duplicates if any **1p**
- Outlier detection and treatment if any **1p**
- Normalization / Standardization if necesarry **1p**
- Feature engineering **3p**
- Train test split, save it.
- Others?


Deliverable:

- Notebook code with no errors.
- Preprocessed data as csv.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
#Uploadam datele
data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = [
    "age", "workclass", "fnlwgt", "education", "education-num", "marital-status",
    "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
    "hours-per-week", "native-country", "income"
]
df = pd.read_csv(data_url, header=None, names=columns, na_values="?", skipinitialspace=True)

In [3]:
#Valori lipsă - pe categorii
for col in ['workclass', 'occupation', 'native-country']:
    df[col] = df[col].fillna(df[col].mode()[0])

#Corectăm și ștergem duplicatele
df.drop_duplicates(inplace=True)

#Ne uităm peste outlieri
#Limităm outlierii folosindu-ne de cuantile pentru a stabiliza modelul
Q1 = df['fnlwgt'].quantile(0.25)
Q3 = df['fnlwgt'].quantile(0.75)
IQR = Q3 - Q1
df['fnlwgt'] = np.where(df['fnlwgt'] > (Q3 + 1.5 * IQR), (Q3 + 1.5 * IQR), df['fnlwgt'])

In [4]:
#Feature engineering
#Calculăm capitalul net
df['capital-net'] = df['capital-gain'] - df['capital-loss']
#Facem bin-uri pentru intervale de vârstă
df['age-group'] = pd.cut(df['age'], bins=[0, 25, 45, 65, 100], labels=['Young', 'Middle', 'Senior', 'Old'])
#Status marital împărțit binar
df['is-married'] = df['marital-status'].apply(lambda x: 1 if str(x).startswith('Married') else 0)

In [5]:
#Codificarea datelor - Excludem tipurile numerice pentru a găsi toate șirurile/categoriile și le codificăm
le = LabelEncoder()
cat_cols = df.select_dtypes(exclude=[np.number]).columns
for col in cat_cols:
    df[col] = le.fit_transform(df[col].astype(str))

# Normalizare și standardizare - Funcții de scalare pentru regresie. Ținta „ore pe săptămână” este păstrată în unitățile originale.
scaler = StandardScaler()
X = df.drop('hours-per-week', axis=1)
y = df['hours-per-week']

X_scaled = scaler.fit_transform(X)
df_preprocessed = pd.DataFrame(X_scaled, columns=X.columns)
df_preprocessed['hours-per-week'] = y.reset_index(drop=True)

In [6]:
#Train Test Split - împărțim în 80% train și 20% test
train_df, test_df = train_test_split(df_preprocessed, test_size=0.2, random_state=42)
#Salvăm în CSV
df_preprocessed.to_csv('preprocessed_census_data.csv', index=False)
#Print-uri de test
print("--- Preprocessing Complete ---")
print(f"Total rows: {len(df_preprocessed)}")
print(f"CSV saved as: preprocessed_census_data.csv")

--- Preprocessing Complete ---
Total rows: 32537
CSV saved as: preprocessed_census_data.csv
